In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import math
import ast
from tqdm import tqdm
import warnings
from pprint import pprint

warnings.filterwarnings("ignore")

In [2]:
all_shots = pd.read_csv("./data/all_shots.csv")

In [3]:
all_shots.shape

(84065, 122)

In [4]:
all_shots.nunique()

season_id                             46
match_id                            3316
ball_receipt_outcome                   0
ball_recovery_recovery_failure         0
block_deflection                       0
                                    ... 
goalkeeper_lost_in_play                0
half_end_early_video_end               0
goalkeeper_saved_to_post               0
goalkeeper_penalty_saved_to_post       0
shot_kick_off                          1
Length: 122, dtype: int64

In [7]:
[column for column in all_shots.columns]

['season_id',
 'match_id',
 'ball_receipt_outcome',
 'ball_recovery_recovery_failure',
 'block_deflection',
 'carry_end_location',
 'clearance_aerial_won',
 'clearance_body_part',
 'clearance_head',
 'clearance_left_foot',
 'clearance_right_foot',
 'counterpress',
 'dribble_nutmeg',
 'dribble_outcome',
 'dribble_overrun',
 'duel_outcome',
 'duel_type',
 'duration',
 'foul_committed_advantage',
 'foul_committed_card',
 'foul_won_advantage',
 'foul_won_defensive',
 'goalkeeper_body_part',
 'goalkeeper_end_location',
 'goalkeeper_outcome',
 'goalkeeper_position',
 'goalkeeper_technique',
 'goalkeeper_type',
 'id',
 'index',
 'interception_outcome',
 'location',
 'minute',
 'off_camera',
 'out',
 'pass_aerial_won',
 'pass_angle',
 'pass_assisted_shot_id',
 'pass_body_part',
 'pass_cross',
 'pass_cut_back',
 'pass_deflected',
 'pass_end_location',
 'pass_goal_assist',
 'pass_height',
 'pass_inswinging',
 'pass_length',
 'pass_outcome',
 'pass_outswinging',
 'pass_recipient',
 'pass_recipien

In [6]:
all_passes = pd.read_csv("./data/all_passes.csv")

In [7]:
all_passes

,season_id,match_id,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,goalkeeper_success_in_play,shot_follows_dribble,half_start_late_video_start,player_off_permanent,pass_backheel,goalkeeper_lost_in_play,half_end_early_video_end,goalkeeper_saved_to_post,goalkeeper_penalty_saved_to_post,shot_kick_off
0,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
687132,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687133,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687134,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
687135,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
all_shots.nunique()

season_id                             46
match_id                            3316
ball_receipt_outcome                   0
ball_recovery_recovery_failure         0
block_deflection                       0
                                    ... 
goalkeeper_lost_in_play                0
half_end_early_video_end               0
goalkeeper_saved_to_post               0
goalkeeper_penalty_saved_to_post       0
shot_kick_off                          1
Length: 122, dtype: int64

In [9]:
all_shots['shot_key_pass_id'].notna().value_counts()

True     59210
False    24855
Name: shot_key_pass_id, dtype: int64

In [10]:
all_shots["related_events"]

0                 ['510415d8-785b-4476-a852-d8dab88514ec']
1                 ['0b3c1ca9-326a-4ad8-b9c4-b615724e8d4b']
2        ['7dfad14b-d366-45bc-9eb2-b0f12c2f76da', '8bdd...
3                 ['a5ca19ae-426d-4751-99c8-026d95033717']
4                 ['286fd384-aa80-4471-9488-98f89fd14af7']
                               ...                        
84060             ['fc529a28-2a80-4a4f-8af2-63f6f9483df1']
84061             ['cbe6239f-1487-497c-9f31-6282d1fefd51']
84062    ['599b62d7-cfe4-44e0-81fa-b4d6e6e8fb24', '5f45...
84063             ['86e3c8ac-41a0-4a2d-b3fc-fa3f6078960c']
84064    ['580e140d-62dc-4c20-ac97-cf357f195af5', 'cd7b...
Name: related_events, Length: 84065, dtype: object

In [11]:
# Function to check if each item in the list is in the column
def get_items_in_column(lst, column):
    found_items = []
    for item in lst:
        if item in column.values:
            found_items.append(item)
    return found_items

# Apply the function to each row in the column
all_shots2 = all_shots.iloc[:20]
temp = all_shots2['related_events'].apply(lambda x: get_items_in_column(x, all_passes['id']))
temp

0     []
1     []
2     []
3     []
4     []
5     []
6     []
7     []
8     []
9     []
10    []
11    []
12    []
13    []
14    []
15    []
16    []
17    []
18    []
19    []
Name: related_events, dtype: object

In [12]:
key_passes = all_passes[all_passes["id"].isin(all_shots['shot_key_pass_id'])]

In [13]:
key_passes

,season_id,match_id,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,goalkeeper_success_in_play,shot_follows_dribble,half_start_late_video_start,player_off_permanent,pass_backheel,goalkeeper_lost_in_play,half_end_early_video_end,goalkeeper_saved_to_post,goalkeeper_penalty_saved_to_post,shot_kick_off
46,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
84,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
234,2015/2016,3890505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
268,2015/2016,3890505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
307,2015/2016,3890505.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
686852,2019,69205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686919,2019,69205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686929,2019,69205.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
686981,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
key_passes[["duration", "pass_angle", "pass_type", "pass_height", "pass_length", "pass_assisted_shot_id"]]

,duration,pass_angle,pass_type,pass_height,pass_length,pass_assisted_shot_id
46,1.707827,-1.850475,Corner,High Pass,39.848340,fd99f442-49ae-4724-87e8-2512e4a5d9c6
84,1.635672,1.716809,Corner,High Pass,37.802250,4aaf9747-c33d-4f76-bb0f-d25ad1ebe7c6
234,1.320689,1.778707,Corner,High Pass,37.303352,749a19ff-d599-4170-a095-79b9865e2b06
268,1.350540,-1.871810,Corner,Low Pass,35.077200,37cdb09d-6504-48cb-a6cc-217790d53b4b
307,1.571437,-1.551568,Recovery,High Pass,10.401923,cffe1d92-0692-47ad-a71b-208c8a52dcc0
...,...,...,...,...,...,...
686852,2.271377,-1.125219,Recovery,Ground Pass,22.274874,2c92a25a-2014-4080-b8fd-b69d72092898
686919,1.689300,0.895606,Free Kick,High Pass,34.717430,828295e4-feac-43c4-8b4a-47617f6d2408
686929,1.788300,-1.394087,Free Kick,High Pass,42.664387,58997323-8e35-474d-815c-7926e1e143ba
686981,1.010200,1.630133,Corner,High Pass,30.353418,4201f68c-764d-45bd-bcfb-96470a72c2e8


In [15]:
key_passes["pass_duration"] = key_passes["duration"]
all_shots = all_shots.drop(["pass_angle", "pass_type", "pass_height", "pass_length", "pass_assisted_shot_id"], axis=1)

all_shots = pd.merge(all_shots, key_passes[["pass_duration", "pass_angle", "pass_type", "pass_height", "pass_length", "pass_assisted_shot_id"]], how='left', left_on='id', right_on='pass_assisted_shot_id', suffixes=("", ""))
all_shots = all_shots.drop("pass_assisted_shot_id", axis=1)

In [16]:
possession_passes = all_passes.groupby(["match_id", "possession"]).agg({
    'match_id': 'first',
    'possession': 'first',
    'duration': 'count'
}).reset_index(drop=True)
possession_passes["num_passes"] = possession_passes["duration"]
possession_passes = possession_passes.drop("duration", axis=1)

all_shots = pd.merge(all_shots, possession_passes, how='left', on=["match_id", "possession"])

In [17]:
all_shots

,season_id,match_id,ball_receipt_outcome,ball_recovery_recovery_failure,block_deflection,carry_end_location,clearance_aerial_won,clearance_body_part,clearance_head,clearance_left_foot,...,half_end_early_video_end,goalkeeper_saved_to_post,goalkeeper_penalty_saved_to_post,shot_kick_off,pass_duration,pass_angle,pass_type,pass_height,pass_length,num_passes
0,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
1,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
2,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
3,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2015/2016,3890561.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84060,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
84061,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,1.2608,1.536767,Throw-in,Low Pass,23.513613,1.0
84062,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.0
84063,2019,69161.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [18]:
# extract relevant attributes from the all_shots dataframe
data = pd.DataFrame()

# time attributes
data["period"] = all_shots["period"]
data["minute"] = all_shots["minute"]
data["possession"] = all_shots["possession"]
data["duration"] = all_shots["duration"]

In [19]:
# qualitative attributes
data["play_pattern"] = all_shots["play_pattern"]
data["position"] = all_shots["position"]

In [21]:
# shot attributes
data["location_x"] = all_shots["location"].apply(lambda x: ast.literal_eval(x)[0])
data["location_x_distance"] = 120 - data["location_x"]
data["location_y"] = all_shots["location"].apply(lambda x: ast.literal_eval(x)[1])
data["location_y_distance"] = abs(data["location_y"] - 40)
data["duration"] = all_shots["duration"].apply(lambda x: x if x < 100 else 0)
data["technique"] = all_shots["shot_technique"]
data["body_part"] = all_shots["shot_body_part"]
data["type"] = all_shots["shot_type"]
data["is_penalty"] = all_shots["shot_type"] == "Penalty"

In [22]:
# shot modifiers
data["first_time"] = all_shots["shot_first_time"].fillna(False)
data["open_goal"] = all_shots["shot_open_goal"].fillna(False)
data["one_on_one"] = all_shots["shot_one_on_one"].fillna(False)
data["aerial_won"] = all_shots["shot_aerial_won"].fillna(False)
data["follows_dribble"] = all_shots["shot_follows_dribble"].fillna(False)
data["under_pressure"] = all_shots["under_pressure"].fillna(False)

In [23]:
# preceding pass attributes
data["pass_duration"] = all_shots["pass_duration"].fillna(all_shots["pass_duration"].mean())
data["pass_angle"] = all_shots["pass_angle"].fillna(all_shots["pass_angle"].mean())
data["pass_type"] = all_shots["pass_type"].fillna("Missing")
data["pass_height"] = all_shots["pass_height"].fillna("Missing")
data["pass_length"] = all_shots["pass_length"].fillna(all_shots["pass_length"].mean())
data["num_passes"] = all_shots["num_passes"].fillna(0)

In [24]:
# defensive/goalkeeper attributes
locations = pd.DataFrame()
locations["opponents"] = all_shots["shot_freeze_frame"].fillna('[{"location": [], "position": {"name": ""}, "teammate": False}]').apply(lambda frame: [[player["location"], player["position"]["name"]] for player in ast.literal_eval(frame) if not player["teammate"]])
locations["shot"] = tuple(zip(data["location_x"], data["location_y"]))
locations["goalkeeper"] = locations["opponents"].apply(lambda frame: [player[0] for player in frame if player[1] == "Goalkeeper"]).apply(lambda x: x[0] if len(x) != 0 else [-1, -1])
locations

,opponents,shot,goalkeeper
0,"[[[115.7, 50.0], Left Back], [[113.2, 39.3], R...","(108.1, 31.2)","[119.0, 38.7]"
1,"[[[98.4, 33.1], Right Defensive Midfield], [[9...","(110.9, 42.6)","[116.0, 42.0]"
2,"[[[118.6, 39.2], Goalkeeper], [[100.4, 44.0], ...","(117.9, 29.1)","[118.6, 39.2]"
3,"[[[93.1, 35.3], Right Wing Back], [[85.8, 34.9...","(101.8, 27.6)","[97.0, 26.8]"
4,"[[[111.2, 32.8], Center Back], [[108.2, 33.2],...","(109.3, 26.5)","[118.3, 36.5]"
...,...,...,...
84060,"[[[119.1, 38.6], Goalkeeper], [[107.3, 47.0], ...","(96.3, 36.2)","[119.1, 38.6]"
84061,"[[[104.9, 30.6], Right Center Back], [[99.6, 2...","(98.5, 24.3)","[119.1, 38.2]"
84062,"[[[79.2, 21.1], Right Center Midfield], [[104....","(102.9, 54.6)","[119.0, 42.5]"
84063,"[[[], ]]","(108.0, 40.0)","[-1, -1]"


In [25]:
def point_in_triangle(point, a, b, c):
    as_x = point[0] - a[0]
    as_y = point[1] - a[1]
    s_ab = ((b[0] - a[0]) * as_y) - ((b[1] - a[1]) * as_x) > 0

    if (((c[0] - a[0]) * as_y) - ((c[1] - a[1]) * as_x) > 0) == s_ab \
            or (((c[0] - b[0]) * (point[1] - b[1])) - ((c[1] - b[1]) * (point[0] - b[0])) > 0) != s_ab:
        return False
    else:
        return True

In [26]:
data["defenders_3m_radius"] = locations.apply(lambda shot: (len([defender for defender in shot["opponents"] if ((defender[0][0]-shot["shot"][0])**2 + (defender[0][1]-shot["shot"][1])**2) < 3**2])) if shot["opponents"] != [[[], '']] else -1, axis=1)
data["defenders_triangle"] = locations.apply(lambda shot: (len([defender for defender in shot["opponents"] if point_in_triangle(defender[0], shot["shot"], [120, 36], [120, 44])])) if shot["opponents"] != [[[], '']] else -1, axis=1)

data["goalkeeper_x"] = locations["goalkeeper"].apply(lambda x: x[0])
data["goalkeeper_y"] = locations["goalkeeper"].apply(lambda x: x[1])
data["distance_to_goalie"] = data.apply(lambda x: math.sqrt((x["goalkeeper_x"] - x["location_x"])**2 + (x["goalkeeper_y"] - x["location_y"])**2), axis=1)

In [27]:
# target variables
data["statsbomb_xg"] = all_shots["shot_statsbomb_xg"].apply(lambda x: float(x))
data["end_location_x"] = all_shots["shot_end_location"].apply(lambda x: ast.literal_eval(x)[0])
data["end_location_y"] = all_shots["shot_end_location"].apply(lambda x: ast.literal_eval(x)[1])
data["is_goal"] = all_shots["shot_outcome"].apply(lambda x: True if x == "Goal" else False)

In [28]:
data["shot_angle"] =  data.apply(lambda x: math.atan2((x["end_location_y"] - x["location_y"]), (x["end_location_x"] - x["location_x"])), axis=1)
data["good_foot"] = data.apply(lambda shot: True if ((shot["body_part"] == "Right Foot" and shot["location_y"] < 42) or (shot["body_part"] == "Left Foot" and shot["location_y"] > 42)) else False, axis=1)

In [29]:
data

,period,minute,possession,duration,play_pattern,position,location_x,location_x_distance,location_y,location_y_distance,...,defenders_triangle,goalkeeper_x,goalkeeper_y,distance_to_goalie,statsbomb_xg,end_location_x,end_location_y,is_goal,shot_angle,good_foot
0,1.0,4.0,7.0,0.597523,Regular Play,Left Wing Back,108.1,11.9,31.2,8.8,...,3,119.0,38.7,13.231024,0.087901,120.0,35.2,False,0.324269,False
1,1.0,6.0,11.0,0.636048,From Throw In,Center Forward,110.9,9.1,42.6,2.6,...,1,116.0,42.0,5.135173,0.160274,120.0,39.3,True,-0.347888,False
2,1.0,8.0,15.0,0.125672,From Throw In,Left Attacking Midfield,117.9,2.1,29.1,10.9,...,1,118.6,39.2,10.124228,0.016036,118.1,30.2,False,1.390943,False
3,1.0,13.0,27.0,1.303079,From Counter,Left Midfield,101.8,18.2,27.6,12.4,...,0,97.0,26.8,4.866210,0.527759,120.0,39.0,True,0.559586,False
4,1.0,17.0,33.0,0.666134,Regular Play,Center Forward,109.3,10.7,26.5,13.5,...,1,118.3,36.5,13.453624,0.074020,118.8,36.0,False,0.785398,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
84060,2.0,59.0,118.0,1.029600,From Free Kick,Right Center Midfield,96.3,23.7,36.2,3.8,...,3,119.1,38.6,22.925968,0.038350,120.0,39.1,False,0.121758,False
84061,2.0,64.0,126.0,1.129000,From Throw In,Left Center Midfield,98.5,21.5,24.3,15.7,...,1,119.1,38.2,24.850956,0.025679,120.0,47.1,False,0.814735,True
84062,2.0,64.0,128.0,1.209200,From Throw In,Right Midfield,102.9,17.1,54.6,14.6,...,1,119.0,42.5,20.140010,0.037896,120.0,45.1,False,-0.507099,False
84063,2.0,75.0,138.0,0.458900,Other,Left Wing,108.0,12.0,40.0,0.0,...,-1,-1.0,-1.0,116.456000,0.783500,120.0,36.3,True,-0.299084,True


In [30]:
data.nunique()

period                     5
minute                   139
possession               290
duration               78058
play_pattern               9
position                  25
location_x               635
location_x_distance      635
location_y               702
location_y_distance      499
technique                  7
body_part                  4
type                       5
is_penalty                 2
first_time                 2
open_goal                  2
one_on_one                 2
aerial_won                 2
follows_dribble            2
under_pressure             2
pass_duration          11912
pass_angle             10519
pass_type                  8
pass_height                4
pass_length             9930
num_passes                13
defenders_3m_radius       10
defenders_triangle        13
goalkeeper_x             258
goalkeeper_y             355
distance_to_goalie     47155
statsbomb_xg           82551
end_location_x           376
end_location_y           705
is_goal       

In [31]:
data.to_csv("./data/augmented_data.csv", index=False)